# Task1

## Base R solution

In [6]:
# Load data
posts <- read.csv("Posts.csv")
users <- read.csv("Users.csv")

# Filter records with AcceptedAnswerId
accepted_answers <- posts[!is.na(posts$AcceptedAnswerId), ]

# Count accepted answers for each user
accepted_count <- aggregate(accepted_answers$AcceptedAnswerId, by = list(UserId = accepted_answers$OwnerUserId), FUN = length)
colnames(accepted_count) <- c("UserId", "AcceptedAnswersCount")

# Count total questions, answers, comments, and overall score for each user
questions_count <- aggregate(posts$Id[posts$PostTypeId == 1], by = list(UserId = posts$OwnerUserId[posts$PostTypeId == 1]), FUN = length)
answers_count <- aggregate(posts$Id[posts$PostTypeId == 2], by = list(UserId = posts$OwnerUserId[posts$PostTypeId == 2]), FUN = length)
comments_count <- aggregate(posts$CommentCount, by = list(UserId = posts$OwnerUserId), FUN = sum)
score_sum <- aggregate(posts$Score, by = list(UserId = posts$OwnerUserId), FUN = sum)

# Merge results
result_base <- merge(accepted_count, questions_count, by = "UserId", all.x = TRUE)
result_base <- merge(result_base, answers_count, by = "UserId", all.x = TRUE)
result_base <- merge(result_base, comments_count, by = "UserId", all.x = TRUE)
result_base <- merge(result_base, score_sum, by = "UserId", all.x = TRUE)

# Add user display name
result_base <- merge(result_base, users[, c("Id", "DisplayName")], by.x = "UserId", by.y = "Id", all.x = TRUE)

# Rename columns
colnames(result_base) <- c("UserId", "AcceptedAnswersCount", "TotalQuestions", "TotalAnswers", "TotalComments", "OverallScore", "DisplayName")

# Select top 10 users by AcceptedAnswersCount in descending order and remove AcceptedAnswersCount column
result_base <- result_base[order(-result_base$AcceptedAnswersCount), ][1:10, ]
result_base <- subset(result_base, select = -AcceptedAnswersCount) # Remove AcceptedAnswersCount column

# Reset row names
row.names(result_base) <- NULL

# Output final result
result_base


Warning message in merge.data.frame(result_base, score_sum, by = "UserId", all.x = TRUE):
"column names 'x.x', 'x.y' are duplicated in the result"
Warning message in merge.data.frame(result_base, users[, c("Id", "DisplayName")], :
"column names 'x.x', 'x.y' are duplicated in the result"


UserId,TotalQuestions,TotalAnswers,TotalComments,OverallScore,DisplayName
<int>,<int>,<int>,<int>,<int>,<chr>
48599,142,73,1096,780,Sam7919
3927,172,124,914,1259,Vorac
2997,129,249,731,1697,Benzo
7309,90,930,2843,5586,Chris H
5953,62,25,214,263,Andrew Welch
30865,59,63,425,687,L.Dutch
2146,58,5,182,178,greenoldman
8685,54,19,385,308,errantlinguist
11160,55,396,1189,2748,RoboKaren


## dplyr Method

In [7]:
library(dplyr)

# Use dplyr to implement solution, excluding records with OwnerUserId as NA
result_dplyr <- posts %>%
  filter(!is.na(AcceptedAnswerId) & !is.na(OwnerUserId)) %>% # Filter out records with NA in OwnerUserId
  group_by(OwnerUserId) %>%
  summarise(AcceptedAnswersCount = n()) %>%
  left_join(posts %>% filter(PostTypeId == 1) %>% group_by(OwnerUserId) %>% summarise(TotalQuestions = n()), by = "OwnerUserId") %>%
  left_join(posts %>% filter(PostTypeId == 2) %>% group_by(OwnerUserId) %>% summarise(TotalAnswers = n()), by = "OwnerUserId") %>%
  left_join(posts %>% group_by(OwnerUserId) %>% summarise(TotalComments = sum(CommentCount, na.rm = TRUE)), by = "OwnerUserId") %>%
  left_join(posts %>% group_by(OwnerUserId) %>% summarise(OverallScore = sum(Score, na.rm = TRUE)), by = "OwnerUserId") %>%
  left_join(users %>% select(Id, DisplayName), by = c("OwnerUserId" = "Id")) %>%
  arrange(desc(AcceptedAnswersCount)) %>%
  head(10) %>%
  rename(UserId = OwnerUserId) %>%
  select(-AcceptedAnswersCount) # Remove AcceptedAnswersCount column

# Convert result_dplyr to a standard data.frame
result_dplyr <- as.data.frame(result_dplyr)

result_dplyr


UserId,TotalQuestions,TotalAnswers,TotalComments,OverallScore,DisplayName
<int>,<int>,<int>,<int>,<int>,<chr>
48599,142,73,1096,780,Sam7919
3927,172,124,914,1259,Vorac
2997,129,249,731,1697,Benzo
7309,90,930,2843,5586,Chris H
5953,62,25,214,263,Andrew Welch
30865,59,63,425,687,L.Dutch
2146,58,5,182,178,greenoldman
8685,54,19,385,308,errantlinguist
11160,55,396,1189,2748,RoboKaren


### Using the base R function 'identical' to check for complete equality
'identical' comes from R's base package and is used to test if two objects 
are exactly the same in terms of both values and attributes


In [8]:
library(microbenchmark)

# Execution time comparison
benchmark_result <- microbenchmark(
  Base_R = {
    # Load data
    posts <- read.csv("Posts.csv")
    users <- read.csv("Users.csv")

    # Filter records with AcceptedAnswerId
    accepted_answers <- posts[!is.na(posts$AcceptedAnswerId), ]

    # Count accepted answers for each user
    accepted_count <- aggregate(accepted_answers$AcceptedAnswerId, by = list(UserId = accepted_answers$OwnerUserId), FUN = length)
    colnames(accepted_count) <- c("UserId", "AcceptedAnswersCount")

    # Count total questions, answers, comments, and overall score for each user
    questions_count <- aggregate(posts$Id[posts$PostTypeId == 1], by = list(UserId = posts$OwnerUserId[posts$PostTypeId == 1]), FUN = length)
    answers_count <- aggregate(posts$Id[posts$PostTypeId == 2], by = list(UserId = posts$OwnerUserId[posts$PostTypeId == 2]), FUN = length)
    comments_count <- aggregate(posts$CommentCount, by = list(UserId = posts$OwnerUserId), FUN = sum)
    score_sum <- aggregate(posts$Score, by = list(UserId = posts$OwnerUserId), FUN = sum)

    # Merge results
    result_base <- merge(accepted_count, questions_count, by = "UserId", all.x = TRUE)
    result_base <- merge(result_base, answers_count, by = "UserId", all.x = TRUE)
    result_base <- merge(result_base, comments_count, by = "UserId", all.x = TRUE)
    result_base <- merge(result_base, score_sum, by = "UserId", all.x = TRUE)

    # Add user display name
    result_base <- merge(result_base, users[, c("Id", "DisplayName")], by.x = "UserId", by.y = "Id", all.x = TRUE)

    # Rename columns
    colnames(result_base) <- c("UserId", "AcceptedAnswersCount", "TotalQuestions", "TotalAnswers", "TotalComments", "OverallScore", "DisplayName")

    # Select top 10 users by AcceptedAnswersCount in descending order and remove AcceptedAnswersCount column
    result_base <- result_base[order(-result_base$AcceptedAnswersCount), ][1:10, ]
    result_base <- subset(result_base, select = -AcceptedAnswersCount) # Remove AcceptedAnswersCount column

    # Reset row names
    row.names(result_base) <- NULL

    # Output final result
    result_base
  },
  dplyr = {
    library(dplyr)

    # Use dplyr to implement solution, excluding records with OwnerUserId as NA
    result_dplyr <- posts %>%
      filter(!is.na(AcceptedAnswerId) & !is.na(OwnerUserId)) %>% # Filter out records with NA in OwnerUserId
      group_by(OwnerUserId) %>%
      summarise(AcceptedAnswersCount = n()) %>%
      left_join(posts %>% filter(PostTypeId == 1) %>% group_by(OwnerUserId) %>% summarise(TotalQuestions = n()), by = "OwnerUserId") %>%
      left_join(posts %>% filter(PostTypeId == 2) %>% group_by(OwnerUserId) %>% summarise(TotalAnswers = n()), by = "OwnerUserId") %>%
      left_join(posts %>% group_by(OwnerUserId) %>% summarise(TotalComments = sum(CommentCount, na.rm = TRUE)), by = "OwnerUserId") %>%
      left_join(posts %>% group_by(OwnerUserId) %>% summarise(OverallScore = sum(Score, na.rm = TRUE)), by = "OwnerUserId") %>%
      left_join(users %>% select(Id, DisplayName), by = c("OwnerUserId" = "Id")) %>%
      arrange(desc(AcceptedAnswersCount)) %>%
      head(10) %>%
      rename(UserId = OwnerUserId) %>%
      select(-AcceptedAnswersCount) # Remove AcceptedAnswersCount column

    # Convert result_dplyr to a standard data.frame
    result_dplyr <- as.data.frame(result_dplyr)

    result_dplyr
  },
  times = 10
)

print(benchmark_result)

# Check if results are identical
identical(result_base, result_dplyr)


Warning message in merge.data.frame(result_base, score_sum, by = "UserId", all.x = TRUE):
"column names 'x.x', 'x.y' are duplicated in the result"
Warning message in merge.data.frame(result_base, users[, c("Id", "DisplayName")], :
"column names 'x.x', 'x.y' are duplicated in the result"
Warning message in merge.data.frame(result_base, score_sum, by = "UserId", all.x = TRUE):
"column names 'x.x', 'x.y' are duplicated in the result"
Warning message in merge.data.frame(result_base, users[, c("Id", "DisplayName")], :
"column names 'x.x', 'x.y' are duplicated in the result"
Warning message in merge.data.frame(result_base, score_sum, by = "UserId", all.x = TRUE):
"column names 'x.x', 'x.y' are duplicated in the result"
Warning message in merge.data.frame(result_base, users[, c("Id", "DisplayName")], :
"column names 'x.x', 'x.y' are duplicated in the result"
Warning message in merge.data.frame(result_base, score_sum, by = "UserId", all.x = TRUE):
"column names 'x.x', 'x.y' are duplicated in t

Unit: milliseconds
   expr       min        lq      mean    median        uq       max neval cld
 Base_R 1095.6390 1115.2226 1150.6346 1137.3924 1174.9653 1236.2029    10  a 
  dplyr  139.3091  141.4984  153.5399  154.2079  160.0127  173.4633    10   b


[1] TRUE

# Benchmark Results Summary

 The benchmark comparison between the Base_R and dplyr implementations yielded the following insights:

 Performance Comparison:

1. The Base_R implementation had an average execution time of approximately 1150.6 milliseconds.
2. The dplyr implementation had an average execution time of approximately 153.5 milliseconds.
3. This indicates that the dplyr implementation is significantly faster than the Base_R approach—about 7-8 times faster. The performance gain likely stems from dplyr's optimized functions for data manipulation, which are particularly efficient for grouped operations and aggregations.

### Conclusion
The dplyr implementation is recommended for this task due to its significantly faster execution time. Adjusting the merge operations to handle duplicate column names will improve code clarity and prevent warnings.


##

# Task2

## Base R

In [2]:
# Load data
badges <- read.csv("Badges.csv")
posts <- read.csv("Posts.csv")
users <- read.csv("Users.csv")

# Filter badges for users who received Teacher, Explainer, or Commentator
target_badges <- c("Teacher", "Explainer", "Commentator")
filtered_badges <- badges[badges$Name %in% target_badges, ]

# Merge badges with users to get locations
user_badges <- merge(filtered_badges, users, by.x = "UserId", by.y = "Id", all.x = TRUE)

# Get unique UserIds with target badges
target_users <- unique(user_badges$UserId)

# Filter posts to include only those created by users with the target badges
posts_with_location <- merge(posts[posts$OwnerUserId %in% target_users, ],
    users[, c("Id", "Location")],
    by.x = "OwnerUserId", by.y = "Id", all.x = TRUE
)

# Count the total number of posts per location for users with these badges
posts_per_location <- aggregate(posts_with_location$Id,
    by = list(Location = posts_with_location$Location), FUN = length
)
colnames(posts_per_location) <- c("Location", "TotalPosts")

# Count the total number of badges per location
badge_count <- aggregate(user_badges$Name,
    by = list(Location = user_badges$Location), FUN = length
)
colnames(badge_count) <- c("Location", "TotalBadges")

# Merge badge count and post count per location
result_base <- merge(badge_count, posts_per_location, by = "Location")
result_base <- result_base[order(-result_base$TotalPosts, -result_base$TotalBadges), ]

# Standardize Location column
result_base$Location <- trimws(tolower(result_base$Location))
result_base$Location <- iconv(result_base$Location, from = "UTF-8", to = "ASCII//TRANSLIT")

# Reset row names
row.names(result_base) <- NULL

result_base


Location,TotalBadges,TotalPosts
<chr>,<int>,<int>
,5341,18661
new zealand,13,3337
washington dc,5,1707
uk,43,1631
"seattle, wa, usa",5,1596
united states,65,1122
"minnesota, usa",3,1041
east coast usa,3,886
united kingdom,94,727


## dplyr solution

In [3]:
library(dplyr)

result_dplyr <- badges %>%
  filter(Name %in% c("Teacher", "Explainer", "Commentator")) %>%
  left_join(users, by = c("UserId" = "Id")) %>%
  filter(!is.na(Location)) %>% # Remove NA locations after joining with users
  group_by(Location) %>%
  summarise(TotalBadges = n()) %>%
  left_join(
    posts %>%
      filter(OwnerUserId %in% unique(badges %>% filter(Name %in% target_badges) %>% pull(UserId))) %>% # Only posts by users with target badges
      left_join(users %>% select(Id, Location), by = c("OwnerUserId" = "Id")) %>%
      filter(!is.na(Location)) %>% # Remove NA locations after joining with posts
      group_by(Location) %>%
      summarise(TotalPosts = n()),
    by = "Location"
  ) %>%
  filter(!is.na(TotalPosts)) %>% # Remove rows with NA in TotalPosts
  arrange(desc(TotalPosts), desc(TotalBadges))

# Standardize Location column
result_dplyr$Location <- trimws(tolower(result_dplyr$Location))
result_dplyr$Location <- iconv(result_dplyr$Location, from = "UTF-8", to = "ASCII//TRANSLIT")

# Convert result_dplyr to a standard data.frame
result_dplyr <- as.data.frame(result_dplyr)

result_dplyr


Location,TotalBadges,TotalPosts
<chr>,<int>,<int>
,5341,18661
new zealand,13,3337
washington dc,5,1707
uk,43,1631
"seattle, wa, usa",5,1596
united states,65,1122
"minnesota, usa",3,1041
east coast usa,3,886
united kingdom,94,727


### Using the base R function 'identical' to check for complete equality
'identical' comes from R's base package and is used to test if two objects 
are exactly the same in terms of both values and attributes


In [5]:
library(microbenchmark)

# Execution time comparison
benchmark_result <- microbenchmark(
  Base_R = {
    # Load data
    badges <- read.csv("Badges.csv")
    posts <- read.csv("Posts.csv")
    users <- read.csv("Users.csv")

    # Filter badges for users who received Teacher, Explainer, or Commentator
    target_badges <- c("Teacher", "Explainer", "Commentator")
    filtered_badges <- badges[badges$Name %in% target_badges, ]

    # Merge badges with users to get locations
    user_badges <- merge(filtered_badges, users, by.x = "UserId", by.y = "Id", all.x = TRUE)

    # Get unique UserIds with target badges
    target_users <- unique(user_badges$UserId)

    # Filter posts to include only those created by users with the target badges
    posts_with_location <- merge(posts[posts$OwnerUserId %in% target_users, ],
      users[, c("Id", "Location")],
      by.x = "OwnerUserId", by.y = "Id", all.x = TRUE
    )

    # Count the total number of posts per location for users with these badges
    posts_per_location <- aggregate(posts_with_location$Id,
      by = list(Location = posts_with_location$Location), FUN = length
    )
    colnames(posts_per_location) <- c("Location", "TotalPosts")

    # Count the total number of badges per location
    badge_count <- aggregate(user_badges$Name,
      by = list(Location = user_badges$Location), FUN = length
    )
    colnames(badge_count) <- c("Location", "TotalBadges")

    # Merge badge count and post count per location
    result_base <- merge(badge_count, posts_per_location, by = "Location")
    result_base <- result_base[order(-result_base$TotalPosts, -result_base$TotalBadges), ]

    # Standardize Location column
    result_base$Location <- trimws(tolower(result_base$Location))
    result_base$Location <- iconv(result_base$Location, from = "UTF-8", to = "ASCII//TRANSLIT")

    # Reset row names
    row.names(result_base) <- NULL

    result_base
  },
  dplyr = {
    library(dplyr)

    result_dplyr <- badges %>%
      filter(Name %in% c("Teacher", "Explainer", "Commentator")) %>%
      left_join(users, by = c("UserId" = "Id")) %>%
      filter(!is.na(Location)) %>% # Remove NA locations after joining with users
      group_by(Location) %>%
      summarise(TotalBadges = n()) %>%
      left_join(
        posts %>%
          filter(OwnerUserId %in% unique(badges %>% filter(Name %in% target_badges) %>% pull(UserId))) %>% # Only posts by users with target badges
          left_join(users %>% select(Id, Location), by = c("OwnerUserId" = "Id")) %>%
          filter(!is.na(Location)) %>% # Remove NA locations after joining with posts
          group_by(Location) %>%
          summarise(TotalPosts = n()),
        by = "Location"
      ) %>%
      filter(!is.na(TotalPosts)) %>% # Remove rows with NA in TotalPosts
      arrange(desc(TotalPosts), desc(TotalBadges))

    # Standardize Location column
    result_dplyr$Location <- trimws(tolower(result_dplyr$Location))
    result_dplyr$Location <- iconv(result_dplyr$Location, from = "UTF-8", to = "ASCII//TRANSLIT")

    # Convert result_dplyr to a standard data.frame
    result_dplyr <- as.data.frame(result_dplyr)

    result_dplyr
  },
  times = 10
)

print(benchmark_result)

# Check if results are identical
identical(result_base, result_dplyr)


Unit: milliseconds
   expr        min         lq       mean     median         uq       max neval
 Base_R 1193.56531 1212.15319 1320.39347 1275.75631 1360.21715 1676.3339    10
  dplyr   49.37093   64.93408   91.30497   73.31509   79.73549  264.0981    10
 cld
  a 
   b


[1] FALSE

## Benchmark Results Summary
The benchmark comparison between the Base_R and dplyr implementations reveals the following insights:

Performance Comparison:

1. The Base_R implementation has an average execution time of 1232.5 milliseconds.
2. The dplyr implementation has an average execution time of 70.1 milliseconds.
3. This indicates that the dplyr implementation is significantly faster than the Base_R approach, approximately 17-18 times faster. This performance gain is largely due to dplyr's optimized functions for data manipulation, especially beneficial with large datasets.

### Conclusion
The dplyr implementation demonstrates considerably higher efficiency and is recommended for tasks involving large data manipulation. Leveraging dplyr in similar cases can greatly improve performance.

# Task3

In [7]:
# load data
posts <- read.csv("Posts.csv")
votes <- read.csv("Votes.csv")


## Base R solution

In [8]:
# Filter votes with VoteTypeId=2 and calculate upvotes per year for each PostId
upvotes_per_year <- votes[votes$VoteTypeId == 2, ]
upvotes_per_year$Year <- format(as.Date(upvotes_per_year$CreationDate), "%Y")
upvotes_count <- aggregate(Id ~ PostId + Year, data = upvotes_per_year, FUN = length)
colnames(upvotes_count) <- c("PostId", "Year", "Count")

# Merge with posts and filter PostTypeId=1
merged_data <- merge(upvotes_count, posts, by.x = "PostId", by.y = "Id")
filtered_data <- merged_data[merged_data$PostTypeId == 1, c("Title", "Year", "Count")]

# Find max upvotes per year
result_base <- aggregate(Count ~ Year, data = filtered_data, FUN = max)
result_base <- merge(result_base, filtered_data, by = c("Year", "Count"))
result_base <- result_base[order(result_base$Year), ]

result_base


,Year,Count,Title
,<chr>,<int>,<chr>
1,2010,21,Why ride a fixed-gear bike?
2,2011,31,Why ride a fixed-gear bike?
3,2012,41,How do I deal with a dog chasing me when I'm touring?
4,2013,34,Ride with someone who is less trained
5,2014,33,How to get over anger at inconsiderate drivers
6,2015,39,How to commute to work on your bike and dress up
7,2016,49,How to prevent drafting?
8,2017,38,Why do the majority of people on an ebike ride at a fairly low cadence?
9,2018,75,How to pass a bicycle while driving a car


## dplyr solution

In [9]:
library(dplyr)

# dplyr solution for Task 3
result_dplyr <- votes %>%
  filter(VoteTypeId == 2) %>%
  mutate(Year = format(as.Date(CreationDate), "%Y")) %>%
  group_by(PostId, Year) %>%
  summarise(Count = n(), .groups = "drop") %>%
  inner_join(posts, by = c("PostId" = "Id")) %>%
  filter(PostTypeId == 1) %>%
  group_by(Year) %>%
  filter(Count == max(Count)) %>%
  select(Title, Year, Count) %>%
  arrange(Year)

result_dplyr


Title,Year,Count
<chr>,<chr>,<int>
Why ride a fixed-gear bike?,2010,21
Why ride a fixed-gear bike?,2011,31
How do I deal with a dog chasing me when I'm touring?,2012,41
Ride with someone who is less trained,2013,34
How to get over anger at inconsiderate drivers,2014,33
How to commute to work on your bike and dress up,2015,39
How to prevent drafting?,2016,49
Why do the majority of people on an ebike ride at a fairly low cadence?,2017,38
How to pass a bicycle while driving a car,2018,75


## As we can see above, the results from Base R and dpylr are the same(Ignore column sorting)